<a href="https://colab.research.google.com/github/futebolista99/PitagoraMaker/blob/master/CC_trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CCのLSTMによる予測

### 1.0 ライブラリのimport

In [ ]:
!pip install japanize_matplotlib
# !pip install janome
# !pip uninstall torchtext
# !pip install torchtext==0.8.1
# !pip install transformers

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import japanize_matplotlib
import math

import numpy as np
import pandas as pd
import spacy
import random
import torch
# import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
# from janome.tokenizer import Tokenizer
from matplotlib.font_manager import FontProperties
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import TensorDataset, DataLoader
# from torchtext.data import Field, LabelField, BucketIterator, TabularDataset
from torchtext import datasets
# from transformers import BertJapaneseTokenizer, BertModel

import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, scale
import pdb
import json

     |████████████████████████████████| 4.1 MB 5.1 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=281682a40b8d0ef9bbd8ab8cee575630bdd9eb60bbba4b8401c94c9da3192fa5
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib


In [ ]:
# デバイスを取得
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {0}".format(DEVICE))

Device: cpu


### 1.1 実装


`LSTM`は、3つのゲートと一つのセルで構成されます。

それぞれを実装する式は次のようになります。($\odot$は要素ごとの積)

(1) 入力ゲート: $\hspace{20mm}\boldsymbol{i}_t = \mathrm{\sigma} \left(\boldsymbol{W}_i \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_i\right)$

(2) 忘却ゲート: $\hspace{20mm}\boldsymbol{f}_t = \mathrm{\sigma} \left(\boldsymbol{W}_f \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_f\right)$  

(3) 出力ゲート: $\hspace{20mm}\boldsymbol{o}_t = \mathrm{\sigma} \left(\boldsymbol{W}_o \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_o\right)$  

(4) セル:　　　 $\hspace{20mm}\boldsymbol{c}_t = \boldsymbol{f}_t \odot \boldsymbol{c}_{t-1} + \boldsymbol{i}_t \odot \tanh \left(\boldsymbol{W}_c \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_c\right)$

(5) 隠れ状態: 　$\hspace{20mm}\boldsymbol{h}_t = \boldsymbol{o}_t \odot \tanh \left(\boldsymbol{c}_t \right)$

`RNN`では各ステップの関数の戻り値は隠れ状態のみ ($\boldsymbol{h}_t$) でしたが、`LSTM`ではセル状態と隠れ状態の2つ ($\boldsymbol{c}_t, \boldsymbol{h}_t$) となるので注意してください。

また、マスクに関してもセルと隠れ状態の両方に適用する必要があります。

In [ ]:
class OurLSTM(nn.Module):
    def __init__(self, hidden_dim, output_dim, attribute_dim):
        super(OurLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        # self.padding_idx = padding_idx

        self.attribute_dim = attribute_dim

        # LSTMの各種パラメータ
        # (1) 入力ゲート
        self.linear_input_gate = nn.Linear(hidden_dim+attribute_dim, hidden_dim)
        
        # (2) 忘却ゲート
        self.linear_forget_gate = nn.Linear(hidden_dim+attribute_dim, hidden_dim)
        
        # (3) 出力ゲート
        self.linear_out_gate = nn.Linear(hidden_dim+attribute_dim, hidden_dim)
        
        # (4) セル
        self.linear_cell = nn.Linear(hidden_dim+attribute_dim, hidden_dim)

        # 最終ステップの隠れ層を入力として受け取る
        self.linear_output = nn.Linear(hidden_dim, output_dim)
        # self.act = nn.Sigmoid()
        # self.act = nn.Softmax()

        
    def forward(self, inputs):  
        # inputs = [max_len, batch_size, attribute_dim]

        batch_size = inputs.shape[1]
        
        # セルと隠れ状態を初期化
        hidden = self.init_hidden(batch_size)  # [batch_size, hidden_dim]
        cell = self.init_cell(batch_size)  # [batch_size, hidden_dim]
        hidden_prev = hidden  # [batch_size, hidden_dim]
        cell_prev = cell  # [batch_size, hidden_dim]

        # # padding対応用のマスクを作成
        max_len = inputs.shape[0]
        
        for i in range(max_len):

            # combined shape: [batch_size, attribute_dim+hidden_dim]
            combined = torch.cat((inputs[i, :, :], hidden_prev), 1)

            # (1) 入力ゲート
            # i_t shape: [batch_size, hidden_dim]
            i_t = torch.sigmoid(self.linear_input_gate(combined))
            
            # (2) 忘却ゲート
            # f_t shape: [batch_size, hidden_dim]
            f_t = torch.sigmoid(self.linear_forget_gate(combined)) 
            
            # (3) 出力ゲート
            # o_t shape: [batch_size, hidden_dim]
            o_t = torch.sigmoid(self.linear_out_gate(combined)) 
            
            # (4) セル
            # c_t shape: [batch_size, hidden_dim]
            c_t = f_t * cell_prev + i_t * torch.tanh(self.linear_cell(combined))
            
            # (5) 隠れ状態
            # h_t shape: [batch_size, hidden_dim]
            h_t = o_t * torch.tanh(c_t)
            
            hidden_prev = h_t  # 隠れ状態の更新
            cell_prev = c_t  # 隠れ状態の更新
            
        # 各文章の最終ステップの隠れ状態を入力 (パディングの一つ前の隠れ状態)
        # output shape: [batch_size, output_dim]
        output = self.linear_output(h_t)
        # 出力をSigmoid関数で変換する
        # output shape: [batch_size]

        # 出力をSoftmax関数で変換する
        # output = self.act(output.squeeze(1))
        
        return output

    # 隠れ状態を初期化する関数
    def init_hidden(self, batch_size):
        hidden = torch.zeros(batch_size, self.hidden_dim, device=DEVICE)
        return hidden

    # Cellを初期化する関数
    def init_cell(self, batch_size):
        cell = torch.zeros(batch_size, self.hidden_dim, device=DEVICE)
        return cell

### 1.2 モデル学習

#### 1.2.1 モデル学習用関数

In [ ]:
def train_model(model, loss_function, optimizer, num_epochs=8):
    # 学習モードに設定
    model.train()

    # モデルの学習
    for epoch in range(num_epochs):
        train_loss, train_acc = 0.0, 0.0
        train_total = 0
        for x, t in tqdm(train_loader):
            x, t = x.to(DEVICE), t.unsqueeze(1).to(DEVICE)
            train_total += x.shape[0]  # テストデータの大きさを取得

            x = torch.transpose(x, 0, 1)
            t = torch.transpose(t, 0, 1)
            optimizer.zero_grad()  # 勾配の初期化          
            t = t.to(torch.int64)

            output = model(x)
            loss = loss_function(output, t[0])  # 損失関数の計算
            train_loss += loss.item()  # 損失の加算

            train_acc += (output.argmax(dim=1) == t[0]).sum().item()  # 正答数の数え上げと可算
            
            loss.backward()  # 勾配の計算(逆伝播)
            optimizer.step()  # パラメータの更新

        avg_train_loss = train_loss / train_total  # 平均損失の計算
        avg_train_acc = train_acc / train_total  # 正答率の計算

        print(('Epoch [{}/{}], train_loss: {train_loss:.5f}, train_acc: {train_acc:.3f}')
        .format(epoch+1, num_epochs, train_loss=avg_train_loss, train_acc=avg_train_acc))

    return model

#### 1.2.2 モデル評価用関数

In [ ]:
def evaluation(model):
    # テストデータの予測
    model.eval()  # 推論モードに切替
    with torch.no_grad():  # 計算グラフの構築をしないよう設定
        # total = x_valid.shape[0]
        test_acc = 0
        valid_total = 0
        for x, t in valid_loader:
            x, t = x.to(DEVICE), t.unsqueeze(1).to(DEVICE)
            valid_total += x.shape[0]  # テストデータの大きさを取得
            x = torch.transpose(x, 0, 1)
            t = torch.transpose(t, 0, 1)
            t = t.to(torch.int64)
            output = model(x)  # 予測の計算
            
            softmax = nn.Softmax(dim=1)
            output = softmax(output)
            test_acc += (output.argmax(dim=1) == t[0]).sum().item()  # 正解数の数え上げと加算
            
            pdb.set_trace()


        print('Test Accuracy: {:.2f} %'.format(100 * test_acc / valid_total))

#### 1.3 データセットとイテレータの定義

In [ ]:
DATA_PATH = '/content/data/'

json_data = pd.read_json(DATA_PATH + 'BTC_USDT-1m-squeeze.json')
json_data.to_csv(DATA_PATH + 'BTC_USDT-1m-squeeze.csv', encoding='utf-8')
csv_data = pd.read_csv(DATA_PATH + 'BTC_USDT-1m-squeeze.csv')

# json_data = pd.read_json(DATA_PATH + 'BTC_USDT-1m.json')
# json_data.to_csv(DATA_PATH + 'BTC_USDT-1m.csv', encoding='utf-8')
# csv_data = pd.read_csv(DATA_PATH + 'BTC_USDT-1m.csv')

csv_data = csv_data.drop(csv_data.columns[0], axis=1)
# cur_data = csv_data.iloc[:, [1, 5]].to_numpy()
cur_data = csv_data.iloc[:, [1, 2, 3, 4, 5]].to_numpy()

# DATA_PATH = '/content/data/'
# json_data = pd.read_json(DATA_PATH + 'BTC_USDT-1m.json')

In [ ]:
# test_x_dim = 1000
# test_y_dim = 1000
# cur_data = np.empty((test_x_dim, test_y_dim))
# test_x = 0
# test_y = 0
# test_z = 0
# for i in range(test_x_dim):
#   for j in range(test_y_dim):
#     # test_z = test_x**7 - 2*test_x**6 + 3*test_x**5 - 4*test_x**4 + 5*test_x**3 - 6*test_x**2 + 7*test_x
#     test_z = 10 * math.exp(-0.1*test_x) * math.cos(3*test_x) + 5 * math.exp(-0.2*test_y) * math.cos(2*test_y)
#     test_z = 2 * math.sin(3*test_x) - 3*math.sin(2*test_y)
#     cur_data[i][j] = test_z
#     test_y += 0.01
#   test_x += 0.01  

In [ ]:
print(cur_data)

[[4.7068500e+04 4.7133200e+04 4.7058500e+04 4.7094020e+04 3.4425658e+01]
 [4.7094030e+04 4.7167660e+04 4.7092170e+04 4.7156650e+04 1.9483291e+01]
 [4.7156640e+04 4.7163300e+04 4.7070900e+04 4.7087060e+04 4.6914043e+01]
 ...
 [4.6339450e+04 4.6383000e+04 4.6336920e+04 4.6375170e+04 2.7507886e+01]
 [4.6375160e+04 4.6428210e+04 4.6350000e+04 4.6389870e+04 7.3221517e+01]
 [4.6389880e+04 4.6416140e+04 4.6354100e+04 4.6361790e+04 2.5874526e+01]]


In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    
    def __init__(self, x_train, t_train):
        self.x_train = x_train
        self.t_train = t_train
        
    def __len__(self):
        return len(self.x_train)
        
    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), \
            torch.tensor(self.t_train[idx], dtype=torch.float)

price_length = 16
attribute_dim = cur_data.shape[1]
change_rate = 0.0003
x_price_list = np.empty((cur_data.shape[0]-price_length, price_length, attribute_dim))
t_price_list = np.empty((cur_data.shape[0]-price_length))

for i in range(cur_data.shape[0]-price_length):
  for j in range(price_length):
    for k in range(attribute_dim):

      x_price_list[i][j][k] = cur_data[i+j][k]

  # pdb.set_trace()    
  if cur_data[i+price_length][0] > (1+change_rate) * cur_data[i+price_length-1][0]:
    t_price_list[i] = 2
  elif cur_data[i+price_length][0] < (1-change_rate) * cur_data[i+price_length-1][0]:
    t_price_list[i] = 0
  else:
    t_price_list[i] = 1

# # 各行でデータを正規化
# for i in range(x_price_list.shape[0]):
#   x_price_list[i] = scale(x_price_list[i])

# x_train, x_valid = \
#     train_test_split(x_price_list, test_size=0.2, random_state=42)

x_train, x_valid, t_train, t_valid = \
    train_test_split(x_price_list, t_price_list, test_size=0.2, random_state=42)

train_dataset = TrainDataset(x_train, t_train)
valid_dataset = TrainDataset(x_valid, t_valid)

# 1.DataLoaderの作成
# 精度向上ポイント: バッチサイズの大小
BATCH_SIZE = 8
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, 
                                           batch_size=BATCH_SIZE)

In [ ]:
print((t_train == 0).sum())
print((t_train == 1).sum())
print((t_train == 2).sum())

print((t_train == 1).sum()/t_train.shape)

320
318
304
[0.33757962]


#### 1.4 学習と評価を実行

In [ ]:
# パラメータの設定

# 隠れ層をxx次元とする。
hidden_dim = 3

# 出力層は1次元(ネガポジ判定のみのため)
output_dim = 3

# モデル定義
our_lstm_model = OurLSTM(hidden_dim, output_dim, attribute_dim).to(DEVICE)
loss_function = nn.CrossEntropyLoss().to(DEVICE)

optimizer = optim.Adam(our_lstm_model.parameters(), lr=0.01)

# モデルの学習（train_model関数を利用）
our_lstm_model = train_model(our_lstm_model, loss_function, optimizer)

# モデルの評価（evaluation関数を利用）
evaluation(our_lstm_model)

100%|██████████| 118/118 [00:00<00:00, 134.97it/s]


Epoch [1/32], train_loss: 0.14144, train_acc: 0.326


100%|██████████| 118/118 [00:00<00:00, 130.13it/s]


Epoch [2/32], train_loss: 0.13790, train_acc: 0.312


100%|██████████| 118/118 [00:00<00:00, 127.91it/s]


...

100%|██████████| 118/118 [00:00<00:00, 129.36it/s]


Epoch [31/32], train_loss: 0.13827, train_acc: 0.317


100%|██████████| 118/118 [00:00<00:00, 129.23it/s]


Epoch [32/32], train_loss: 0.13796, train_acc: 0.329
> <ipython-input-56-d4d3fc7d484a>(8)evaluation()
-> for x, t in valid_loader:
(Pdb) p output
tensor([[0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380],
        [0.3363, 0.3256, 0.3380]])
(Pdb) q


BdbQuit: ignored

#### 補足 CrossEntropyLossの使い方

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(2, 5, requires_grad=True)
target = torch.empty(2, dtype=torch.long).random_(5)
output = loss(input, target)

# exp_input = torch.exp(input)

In [ ]:
print(input)
# print(exp_input)
print(target)
print(output)

tensor([[-0.7185, -0.2972,  0.6030, -2.4340,  0.3861],
        [ 0.3964, -1.0939,  2.6953,  0.3969, -0.3262]], requires_grad=True)
tensor([2, 3])
tensor(1.7329, grad_fn=<NllLossBackward0>)


In [ ]:
row1 = -1 * input[0, 2] + math.log(math.exp(input[0, 0])+math.exp(input[0, 1])+math.exp(input[0, 2])+math.exp(input[0, 3])+math.exp(input[0, 4]))
row2 = -1 * input[1, 3] + math.log(math.exp(input[1, 0])+math.exp(input[1, 1])+math.exp(input[1, 2])+math.exp(input[1, 3])+math.exp(input[1, 4]))
print(row1 + row2)


tensor(3.4658, grad_fn=<AddBackward0>)


In [ ]:
1.7329*2

3.4658

#### 1.2.3 予測結果の確認

学習した`LSTM`のモデルを用いて、各文章が`positive`または`negative`のどちらに分類されたかを確認してみます。

In [ ]:
# ID化されたトークンを自然言語の文章に変換する関数
def token2text(tokens, TEXT):
    texts = []
    for token in tokens:
        text = TEXT.vocab.itos[token]
        if text != "<pad>":
            texts.append(text)
        else:
            break
    return "".join(texts)


# data_numで指定した数の例をテストデータから取り出し、出力結果を表示する関数
def print_result(model, data_num=5):
    # 出力例を確認
    model.eval()  # 推論モードに切替
    batch = next(iter(val_iter))  
    predicts = model(batch.text)  # 予測の計算
    predicts = torch.round(predicts)  # 予測値をラベルに変換
    for i in range(data_num):
        tokens = batch.text[0][:, i]
        print("input text: {}".format(token2text(tokens, TEXT)))
        print("answer label: {}".format(LABEL.vocab.itos[batch.label[i]]))
        print("predicted label: {}\n".format(LABEL.vocab.itos[int(predicts[i])]))

In [ ]:
# 出力例を確認（print_result関数を利用）
print_result(our_lstm_model)

input text: ｆａ部門の連結売上高は、1,750億16百万円（前期比2.8％増)、全連結売上高に対する構成比は32.6％となりました
answer label: positive
predicted label: positive

input text: 当社グループを取り巻くエネルギー業界においては、省エネルギー化や顧客ニーズの多様化などにより石油製品の需要が減少傾向にあるなか、石油<unk>の再編への動きが進むとともに、昨年４月に電力の<unk>が全面自由化された
answer label: negative
predicted label: negative

input text: 経常損益につきましては、営業外収益に為替差益、営業外費用に支払利息を計上したこと等により、経常利益は12億71百万円（前年同期５億89百万円、115.6％増）となりました
answer label: positive
predicted label: positive

input text: このような環境のもと、当事業年度における売上高は、前年比5.6％減の4,115,<unk>千円（うち輸出1,<unk>,<unk>千円　全売上高の25.8％）と２年連続の減収となりました
answer label: negative
predicted label: negative

input text: 土地の<unk>売却が減少したこと等により減収となりましたが、分譲マンションにおいて高価格帯物件が増加したこと等により売上が増加、粗利益率も改善したことにより増益となりました
answer label: positive
predicted label: positive



### 1.3 より実践的なLSTMの実装

`LSTM`は`PyTorch`で用意されている`torch.nn.LSTM`を用いて実装することもできます。

この実装方法では、モデルの学習時に各データを明示的に`loop`で回して実行する必要がないため、より簡単に実装することができます。

そのため、実務では一般的にこちらの実装方法が用いられます。　参考までに、`toch.nn.LSTM`の実装例を示します。

`torch.nn.LSTM`で主に使うパラメータは以下の5つです。他にもいくつか設定できるパラメータがあるので、必要に応じてドキュメントを確認してみてください。

```
input_size : 入力の次元数
hidden_size : 隠れ状態の次元数
num_layers : LSTMの層数
dropout : 各LSTMの最終層のDropout Rate
bidirectional : 双方向のLSTMを用いるかどうか (【発展】 Bidirectional LSTMの章で紹介します)
```

参考：https://pytorch.org/docs/master/generated/torch.nn.LSTM.html

#### 1.3.1 nn.LSTMを用いてモデルを定義

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = 1
        
        # 入力単語をEmbeddingする
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        
        # LSTMを定義する
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.num_layers, bidirectional=False)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, inputs):
        # inputs = (text, text_length)
        inputs_text = inputs[0]  # 入力テキストを取得、 inputs_text = [sentence lengths, batch_size]
        inputs_length = inputs[1]  # 入力テキストの文字列長を取得

        # 文章内の各単語をEmbeddingする
        # embedded shape: [sentence lengths, batch_size, embedding_dim]
        embedded = self.embeddings(inputs_text)
        # paddingされた単語で状態が更新されないようにする
        packed_embedded = pack_padded_sequence(embedded, inputs_length, enforce_sorted=False) 
        
        # LSTMに入力して各ステップで状態を更新
        # output: 前ステップでの状態
        # hidden: 最終ステップでの隠れ状態
        # cell: 最終ステップでのセル
        output, (hidden, cell) = self.lstm(packed_embedded)
        
        # LSTMの最終ステップの隠れ状態を入力する
        # output shape: [batch_size, output_dim]
        output = self.linear(hidden.squeeze(0))
        # 出力をSigmoid関数で変換する
        # output shape: [batch_size]
        output = self.act(output.squeeze(1))
        return output

#### 1.3.2 モデルの学習

In [ ]:
# パラメータの設定
vocab_size = len(TEXT.vocab)
embedding_dim = 512
hidden_dim = 256
padding_idx = TEXT.vocab.stoi["<pad>"]
output_dim = 1

# モデル定義
lstm_model = LSTM(vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx).to(DEVICE)
loss_function = nn.BCELoss().to(DEVICE)
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# モデルの学習
lstm_model = train_model(lstm_model, loss_function, optimizer)

# モデルの評価
evaluation(lstm_model)

#### 1.3.3 予測結果の確認

In [ ]:
# 出力例を確認
print_result(lstm_model)

---

## 【補足】Gradient Clipping（長系列への対処法）

`LSTM`は長系列に対しても学習がうまく行きやすいモデルでしたが、一般の`RNN`における長系列の学習のTipsとして、**Gradient Clipping**に触れておきます。

`RNN`では誤差逆伝播法が特に**Back Propagation Through Time (BPTT)**と呼ばれるものになり、各層のみならず各時点の勾配が乗算されます。

そのため、通常よりも勾配が過大（或いは過小）になりやすいという特徴をもっています。

こうした現象を**勾配爆発（消失）**と呼びますが、勾配爆発は学習を不安定化し収束を困難にします。

![Clipping](/figures/Clipping.png)

出典：Ian Goodfellow et. al, “Deep Learning”, MIT press, 2016 (http://www.deeplearningbook.org/)

そこで、勾配の大きさを意図的に制限して対処しようというのが、`Gradient Clipping`と呼ばれる手法です。

以下のように、明示的に`optimizer`から勾配を取得した後、`torch.nn.utils.clip_grad_norm`関数に通した上で勾配を適用することで実行できます。

In [ ]:
# トレーニング時の各ステップでloss.backward()の後に以下を記述する。
# 例としてモデルの定義名がlstmの場合を示す。

clipping_value = 1  # 許容する勾配の最大値を設定する
torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), clipping_value)

### 【発展】 GRU (Gated Recurrent Unit)

`GRU (Gated Recurrent Unit)`は`LSTM`を単純化した構造を持ちます。そのため、モデルの表現力は`LSTM`と比較して劣りますが実行速度が速いと言った利点があります。

`GRU`も`PyTorch`から`torch.nn.GRU`が用意されているため、簡単に実装することができます。

`torch.nn.GRU`で主に使うパラメータは以下の5つです。他にもいくつか設定できるパラメータがあるので、必要に応じてドキュメントを確認してみてください。

```
input_size : 入力の次元数
hidden_size : 隠れ状態の次元数
num_layers : LSTMの層数
dropout : 各LSTMの最終層のDropout Rate
bidirectional : 双方向のGRUを用いるかどうか
```

参考 : https://pytorch.org/docs/master/generated/torch.nn.GRU.html

#### nn.GRUを用いてモデルを定義

In [ ]:
class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = 1
        
        # 入力単語をEmbeddingする
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        
        # GRUを定義する
        # nn.GRU以外はLSTMの時と同様で利用できる
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=self.num_layers, bidirectional=False)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, inputs):
        # inputs = (text, text_length)
        inputs_text = inputs[0]  # 入力テキストを取得、 inputs_text = [sentence lengths, batch_size]
        inputs_length = inputs[1]  # 入力テキストの文字列長を取得

        # 文章内の各単語をEmbeddingする
        # embedded shape: [sentence lengths, batch_size, embedding_dim]
        embedded =  # WRITE ME
        # paddingされた単語で状態が更新されないようにする
        packed_embedded =  # WRITE ME
        
        # GRUに入力して各ステップで状態を更新
        # output: 前ステップでの状態
        # hidden: 最終ステップでの隠れ状態
        output, hidden =  # WRITE ME
        
        # GRUの最終ステップの隠れ状態を入力する
        # output shape: [batch_size, output_dim]
        output =  # WRITE ME
        # 出力をSigmoid関数で変換する
        # output shape: [batch_size]
        output =  # WRITE ME
        return output

#### モデルの学習

In [ ]:
# パラメータの設定
vocab_size = len(TEXT.vocab)
embedding_dim = 512
hidden_dim = 256
padding_idx = TEXT.vocab.stoi["<pad>"]
output_dim = 1

# モデル定義
gru_model = GRU(vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx).to(DEVICE)
loss_function = nn.BCELoss().to(DEVICE)
optimizer = optim.Adam(gru_model.parameters(), lr=0.001)

# モデルの学習
gru_model = train_model(gru_model, loss_function, optimizer)

# モデルの評価
evaluation(gru_model)

#### 予測結果の確認

In [ ]:
# 出力例を確認
print_result(gru_model)

### 【発展】 Bidirectional LSTM

`LSTM`を定義する時に、引数`bidirectional`の値を`True`に設定することで`Bidirectional LSTM` (双方向LSTM)を用いることができます。

`Bidirectional LSTM`では、過去から未来方向へのタイムステップだけでなく、未来から過去方向への学習も行います。そのため、通常の`LSTM`より文脈をうまく捉えることができベンチマークなどで高いスコアが出ています。

#### Bi-LSTMの定義

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx):
        super(BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = 1

        # 入力単語をEmbeddingする
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)

        # LSTMを定義する
        # bidirectionalの値をTrueに設定することでBidirectional LSTMを用いることができる
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.num_layers, bidirectional=True)
        self.linear = nn.Linear(hidden_dim*2, output_dim)  # 2方向になるので隠れ状態も2倍になる
        self.act = nn.Sigmoid()
        
    def forward(self, inputs):
        # inputs = (text, text_length)
        inputs_text = inputs[0]  # 入力テキストを取得、 inputs_text = [sentence lengths, batch_size]
        inputs_length = inputs[1]  # 入力テキストの文字列長を取得

        # 文章内の各単語をEmbeddingする
        # embedded shape: [sentence lengths, batch_size, embedding_dim]
        embedded =  # WRITE ME
        # paddingされた単語で状態が更新されないようにする
        packed_embedded =  # WRITE ME
        
        # Bi-LSTMに入力して各ステップで状態を更新
        # output: 前ステップでの状態
        # hidden: 最終ステップでの隠れ状態
        # cell: 最終ステップでのセル
        output, (hidden, cell) =  # WRITE ME
        
        # LSTMの最終ステップの隠れ状態を入力する
        # 2方向LSTMの最終ステップの各隠れ状態連結して入力する
        # output shape: [batch_size, output_dim]
        output =  # WRITE ME
        # 出力をSigmoid関数で変換する
        # output shape: [batch_size]
        output =  # WRITE ME
        return output

#### モデルの学習

In [ ]:
# パラメータの設定
vocab_size =  # WRITE ME
embedding_dim =  # WRITE ME
hidden_dim =  # WRITE ME
padding_idx =  # WRITE ME
output_dim =  # WRITE ME

# モデル定義
bi_lstm_model =  # WRITE ME
loss_function =  # WRITE ME
optimizer =  # WRITE ME

# モデルの学習（train_model関数を利用）
bi_lstm_model =  # WRITE ME

# モデルの評価（evaluation関数を利用）
# WRITE ME

#### 予測結果の確認

In [ ]:
# 出力例を確認（print_result関数を利用）
# WRITE ME

---

## 課題2. Sequence Model

### 1. データセットの読み込みと単語・品詞のID化

今回はデータセットとして、英文とその日本語対訳がセットになった Tanaka Corpus ( http://www.edrdg.org/wiki/index.php/Tanaka_Corpus ) を使用します。

（厳密にはデータサイズの関係で、Tanaka Corpusの一部を抽出した https://github.com/odashi/small_parallel_enja を使っています。）

train.enとtrain.jaの中身は次のようになっています。

- train.enの中身 (英語の文)
```
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
︙
```

- train.jaの中身(日本語の文、対訳)
```
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
︙
```

この通り、そのまま読み込むと単語のまま読み込まれます。

これでは計算的には扱いづらいので、第7章の時と同様にそれぞれの単語を**数字によるIDに置き換え**ましょう。

この数値化には、`janome.tokenizer`の`Tokenizer`を用います。 `index=0`は、未知語を表す`<unk>`に割り当てられ、

`index=1`はpaddingを表す`<pad>`に割り当てられます。

ここではTokenizerの仕様については深く立ち入りませんので、気になる方は公式ドキュメントをチェックしてください。

なお、読み込む際には

- 文頭を表す仮想単語（**SOS**, Start Of Sentence）として`<sos>`

- 文末を表す仮想単語（**EOS**, End Of Sentence）として`<eos>`
    
を付加します。

参考： https://mocobeta.github.io/janome/

In [ ]:
# 日本語用のtokenizer
ja_t = Tokenizer()
def tokenizer_ja(text): 
    return  # WRITE ME

# 英語用のtokenizer
spacy_en = spacy.load('en')
def tokenizer_en(text):
    return  # WRITE ME (spacy_en.tokenizer()を利用)

# 各種Fieldを定義
# 文章の始まりを表す<sos>と文章の終わりを表す<eos>を各文章に付与する
SOURCE = Field(sequential=True, tokenize=tokenizer_en, init_token="<sos>", eos_token="<eos>", lower=True, include_lengths=True)
TARGET = Field(sequential=True, tokenize=tokenizer_ja, init_token="<sos>", eos_token="<eos>", lower=False, include_lengths=True)

train, val = TabularDataset.splits(
    path="/root/userspace/public/day5/chap10/data/",
    train="train.csv", validation="val.csv",
    format="csv", skip_header=True,
    fields=[("ids", None), ("source", SOURCE), ("target", TARGET)]
)


# SOURCEとTARGETそれぞれの単語に番号を振る
# 最低出現回数をmin_freqで指定
# WRITE ME (SOURCE側)
# WRITE ME (TARGET側)

# イテレータの作成
batch_size = 128
train_iter, val_iter = BucketIterator.splits(
    (train, val), batch_size=batch_size, device=DEVICE, sort=False
)

### 2. 各層クラスの実装

`Encoder-Decoder`モデルでは、入力系列 (`Source`)を`Encoder`を用いて固定長のベクトルに変換し、得られたベクトルを`Decoder`に入力し出力系列 (`Target`)に近くなるように系列を生成します。

また、生成を行う際には1ステップずつ逐次的に`LSTM`を実行したいので、状態を保持する機能も持たせる必要があります。

はじめに、`Encoder`と`Decoder`をそれぞれ定義します。　その後、それらを組み合わせることで翻訳を行う`Seq2Seq`モデルを定義します。

In [ ]:
# Encoder側の定義
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, n_layers, dropout, padding_idx):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.padding_idx = padding_idx
        
        self.embed =  # WRITE ME
        
        # 今回はtorch.nn.LSTMを用いて実装してください
        self.lstm =  # WRITE ME


    def forward(self, src_text, src_length):
        embedded = self.embed(src_text)
        # paddingされた単語で状態が更新されないようにする
        packed_embedded = pack_padded_sequence(embedded, src_length, enforce_sorted=False)
        
        # embeddingの結果をLSTMに入力
        outputs, (hidden, cell) =  # WRITE ME
        outputs, sent_len = pad_packed_sequence(outputs)
        
        return outputs, hidden, cell, sent_len

    
# Decoder側の定義
class Decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, n_layers, output_size, dropout, padding_idx):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.padding_idx = padding_idx

        self.embed =  # WRITE ME
        self.dropout = nn.Dropout(dropout)
        
        # 今回はtorch.nn.LSTMを用いて実装してください
        self.lstm =  # WRITE ME
        
        # 線形層を用いて、hidden_size -> output_sizeの空間に射影してください
        self.fc_out =  # WRITE ME

    def forward(self, input, hidden, cell):
        embedded = self.embed(input).unsqueeze(0) 
        embedded = self.dropout(embedded)

        # embeddingの結果をLSTMに入力
        _, (hidden, cell) =  # WRITE ME
        prediction = self.fc_out(hidden.squeeze(0))

        return prediction, hidden, cell

今回は以下のようなモデルを実装してみましょう。（ここでは、図中の`hidden layer 2`は省略します。）

<img src="./figures/seq2seq.png" width="50%">

引用：https://github.com/tensorflow/nmt

モデルの構造としては、

- 翻訳したい文をRNNで状態ベクトルによる表現に変換する（**Encoder**）
- 上述の状態ベクトルを考慮しつつ、**訳文の1単語先を予測**する（**Decoder**）

という大きく2つの部分からなっています。

学習タスクとしては、訳文の時点$t$の単語から時点$t+1$の単語の予測タスクとなっていますが、翻訳前の文章も考慮するわけです。

なお、誤差関数は多クラス交差エントロピーで、具体的には次のようになります。（ミニバッチサイズ：$N$、系列長：$T$、辞書サイズ：$K$）

$$
    \mathrm{E}\left(\{\boldsymbol{S}^{(n)}\}_{n=1,\ldots,N},\{\boldsymbol{Y}^{(n)}\}_{n=1,\ldots,N}\right) = -\frac{1}{N}\sum^N_{n=1}\sum^T_{t=1}\sum^K_{k=1} s^{(n)}_{t, k} \log y^{(n)}_{t, k}
$$

$$\left(\boldsymbol{S}^{(n)} = [\boldsymbol{s}^{(n)}_1\ \ \boldsymbol{s}^{(n)}_2\ \ \cdots \ \ \boldsymbol{s}^{(n)}_T]\in\{0,1\}^{K \times T},\ \ \boldsymbol{Y}^{(n)} = [\boldsymbol{y}^{(n)}_1\ \ \boldsymbol{y}^{(n)}_2\ \ \cdots \ \ \boldsymbol{y}^{(n)}_T]\in\mathbb{R}^{K \times T}\ \ \ n=1,2,\ldots,N\right)$$

$\boldsymbol{s}^{(n)}_t$は$n$番目の系列の時点$t$での単語のone_hot表現です。

また、各$\boldsymbol{y}^{(n)}_t$はモデルからの出力なので、モデルのパラメータ$\boldsymbol{\theta}$に依存しており, 最適化はこの$\boldsymbol{\theta}$について行われます。

In [ ]:
# 定義したEncoderとDecoderを用いてSeq2Seqモデルを定義する
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        src_text = src[0]  # Sourceのテキストを取得
        src_length = src[1]  # Sourceのテキストを取得
        trg_text = trg[0]  # Targetのテキストを取得
        trg_length = trg[1]  # Targetのテキスト長を取得
        batch_size = trg_text.shape[1]  # Batch Sizeを取得
        max_trg_len = trg_text.shape[0]  # Targetの最大文字列長を取得
        trg_vocab_size = self.decoder.output_size  # TargetのVocaburay数を取得

        # Decoderの出力結果を保持する配列
        outputs = torch.zeros(max_trg_len, batch_size, trg_vocab_size).to(DEVICE)

        # Encoderの最後の隠れ状態をDecoderの初期値とする
        _, encoder_hidden, encoder_cell, _ = self.encoder(src_text, src_length)

        # 文章の始まりを表す<sos>トークンをDecoderに入力する
        input = trg_text[0, :]

        # 各ステップを実行
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell
        for step in range(1, max_trg_len):
            output, decoder_hidden, decoder_cell =  # WRITE ME
            outputs[step] = output

            # 予測結果の配列から、もっとも確率が高い予測トークンを取得する
            # output = [0.1, 0.1, 0.05, ...., 0.7, 0.05]
            top1 =  # WRITE ME

            # 次のstepでの入力を更新
            input = top1

        return outputs

### 3. 学習

#### モデル学習用の関数

In [ ]:
def train_seq2seq_model(model, loss_function, optimizer, num_epochs=10, clip=1.0):
    # 学習モードに設定
    model.train()

    for epoch in range(num_epochs):
        train_loss = 0.0
        for batch in tqdm(train_iter):
            optimizer.zero_grad()  # 勾配の初期化
            output = model(batch.source, batch.target)  # 予測の計算（順伝播）
            
            # 予測結果の整形と正解データの抽出
            output_size = output.shape[-1]
            output = output[1:].view(-1, output_size)  # <sos>を除く
            
            trg_text = batch.target[0]
            trg = trg_text[1:].view(-1)  # <sos>を除く
            
            loss = loss_function(output, trg)  # 損失関数の計算
            train_loss += loss.item()  # 損失の加算
            loss.backward()  # 勾配の計算(逆伝播)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  # Gradient Clipping
            optimizer.step()  # パラメータの更新
            
        avg_train_loss = train_loss / len(train)  # 平均損失の計算
        print(('Epoch [{}/{}]: train_loss: {train_loss:.5f}')
          .format(epoch+1, num_epochs, train_loss=avg_train_loss))
        
    return model

#### モデル評価用の関数

In [ ]:
def evaluate_seq2seq_model(model, loss_function):
    model.eval()  # 推論モードに切替
    with torch.no_grad():  # 計算グラフの構築をしないよう設定
        test_loss = 0
        for batch in tqdm(val_iter):
            output = model(batch.source, batch.target)  # 予測の計算（順伝播）

            # 予測結果の整形と正解データの抽出
            output_size = output.shape[-1]
            output = output[1:].view(-1, output_size)  # <sos>を除く
            
            trg_text = batch.target[0]
            trg = trg_text[1:].view(-1)  # <sos>を除く

            loss = loss_function(output, trg)  # 損失関数の計算
            test_loss += loss.item()

        avg_test_loss = test_loss / len(val)
        print('Test Loss: {test_loss:.5f}'.format(test_loss=avg_test_loss))

#### 学習と評価を実行

In [ ]:
# ハイパーパラメータの設定
input_size = len(SOURCE.vocab)
output_size = len(TARGET.vocab)
encoder_embed_size = 256
decoder_embed_size = 256
hidden_size = 512
n_layers = 1
encoder_dropout = 0.5
decoder_dropout = 0.5
src_padding_idx = SOURCE.vocab.stoi["<pad>"]
trg_padding_idx = TARGET.vocab.stoi["<pad>"]

# EncoderクラスとDecoderクラスを生成
encoder = Encoder(input_size, encoder_embed_size, hidden_size, n_layers, encoder_dropout, src_padding_idx)
decoder = Decoder(decoder_embed_size, hidden_size, n_layers, output_size, decoder_dropout, trg_padding_idx)

# デバイスを指定して翻訳モデルを生成
seq2seq_model = Seq2Seq(encoder, decoder).to(DEVICE)
optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss(ignore_index=trg_padding_idx)

# モデルのパラメータを初期化
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
seq2seq_model.apply(init_weights)

# モデルの学習（train_seq2seq_model関数を利用）
seq2seq_model =  # WRITE ME

# モデルの評価（evaluate_seq2seq_model関数を利用）
# WRITE ME

### 4. 生成

トレーニングデータで学習した`Seq2Seq`モデルを用いて、テストデータの出力結果を確認してみます。

#### 正解の文章と生成された文章を確認する関数

In [ ]:
# ID変換された各単語のリストを文章に変換する
def token2text(tokens, VOCAB):
    texts = []
    for token in tokens[1:]:  # <sos>を除く
        text = VOCAB.vocab.itos[token]
        if text != "<eos>":
            texts.append(text)
        else:
            break
       
    return " ".join(texts)

# 出力例を確認
def print_seq2seq_result(model):
    model.eval()  # 推論モードに切替
    batch = next(iter(val_iter))
    predicts = model(batch.source, batch.target)  # 予測の計算
    predicts = predicts.argmax(2)  # 予測値のみを取得: (N, T, V) =>  (N, T)
    for i in range(5):
        source = batch.source[0][:, i]
        target = batch.target[0][:, i]
        print("English (input): {0}".format(token2text(source, SOURCE)))
        print("Japanese (answer): {0}".format(token2text(target, TARGET)))
        print("predicted: {0}\n".format(token2text(predicts[:, i], TARGET)))

#### 生成された文章の確認

In [ ]:
# 出力例を確認（print_seq2seq_result関数を利用）
# WRITE ME

### 【補足】Teacher Forcing

`Teacher Forcing`とは、モデルの学習時に`Decoder`の入力として1ステップ前の`Decoder`の出力結果を用いるのではなく、

正解データのターゲット配列をそのまま用いる手法です。　

このようにすることで、モデルの学習が安定し収束が早くなるというメリットがあります。

先ほど実装した`Seq2Seq`クラスに3点変更を加えることで、`teacher forcing`を用いた日英翻訳モデルを作成することができます。

In [ ]:
# Teacher Forcingを用いたSeq2Seqモデルを定義する
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    # [変更箇所 1] 引数にteacher forcingを用いる比率を追加する
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        src_text = src[0]  # Sourceのテキストを取得
        src_length = src[1]  # Sourceのテキストを取得
        
        trg_text = trg[0]  # Targetのテキストを取得
        trg_length = trg[1]  # Targetのテキスト長を取得
        
        batch_size = trg_text.shape[1]  # Batch Sizeを取得
        
        max_trg_len = trg_text.shape[0]  # Targetの最大文字列長を取得
        trg_vocab_size = self.decoder.output_size  # TargetのVocaburay数を取得

        # Decoderの出力結果を保持する配列
        outputs = torch.zeros(max_trg_len, batch_size, trg_vocab_size).to(DEVICE)

        # Encoderの最後の隠れ状態をDecoderの初期値とする
        _, encoder_hidden, encoder_cell, _ = self.encoder(src_text, src_length)

        # 文章の始まりを表す<sos>トークンをDecoderに入力する
        input = trg_text[0, :]

        # 各ステップを実行
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell
        for step in range(1, max_trg_len):
            output, decoder_hidden, decoder_cell = self.decoder(input, decoder_hidden, decoder_cell)
            outputs[step] = output
            
            # [変更箇所 2] 現在のステップでteacher forcingを用いるかどうか
            teacher_force =  # WRITE ME (random.random()を利用)

            # 予測結果の配列から、もっとも確率が高い予測トークンを取得する
            # output = [0.1, 0.1, 0.05, ...., 0.7, 0.05]
            top1 = output.argmax(1)

            # [変更箇所 3] techer forceingを用いる場合 (teacher_foce=True)は、
            # target配列から該当する値を抽出し、用いない場合 (teacher_foce=False)は
            # 予測結果 (top1)を次のステップでの入力とする
            input =  # WRITE ME

        return outputs

上記で学習に用いていた`Seq2Seq`の代わりに、新たに作成した`Teach Forcing`を用いた`Seq2Seq`のモデルを指定することで、コードを変更することなく動作確認を行うことができます。

余力がある場合は、`Teach Forcing`を用いた場合とそうでない場合で収束にかかる時間やモデルの精度を比較してみてください。

---

## 課題3. Attention Model

### 1. Attention Modelの概要

今回は Luong et al., 2015のGlobal attentionモデルを実装します。

- "Effective Approaches to Attention-based Neural Machine Translation", Minh-Thang Luong et al., EMNLP 2015 https://arxiv.org/abs/1508.04025

課題2で実装したモデルは左図、ここで実装するモデルは右図になります。

<img src="./figures/attention-1.png" width="1000mm">

#### Decoderの各ステップにおける計算の手順

`Encoder`の各ステップの隠れ層を

$$
    \boldsymbol{\bar{h}} = \{\boldsymbol{\bar{h}}_1, \boldsymbol{\bar{h}}_2, \ldots, \boldsymbol{\bar{h}}_s, \ldots, \boldsymbol{\bar{h}}_S\}
$$

`Decoder`の各ステップの隠れ層を

$$
    \boldsymbol{h} = \{\boldsymbol{h}_1, \boldsymbol{h}_2, \ldots, \boldsymbol{h}_t, \ldots, \boldsymbol{h}_T\}
$$

とします。

`Attention Layer`の計算手順は以下のようになります。

1. Decoderの各時点の出力に対して、入力系列のどのステップに注目するかを表すscore関数の値を計算します。
$$
   \mathrm{score}\left(\boldsymbol{\bar{h}}_s,\ \boldsymbol{h}_t\right) = \boldsymbol{h}_t^{\mathrm{T}} \boldsymbol{W}_a \boldsymbol{\bar{h}}_s
$$
なお、score関数には任意性があり、今回用いる関数以外にも例えば以下の様なものが提案されています。
$$
\mathrm{score}\left(\boldsymbol{\bar{h}}_s,\ \boldsymbol{h}_t\right) =
\begin{cases}
    {\boldsymbol{h}_t}^{\mathrm{T}} \boldsymbol{\bar{h}}_s \\
    \boldsymbol{v}^{\mathrm{T}} \tanh\left(\boldsymbol{W}_{ad} \boldsymbol{h}_t + \boldsymbol{W}_{ae} \boldsymbol{\bar{h}}_s\right)
\end{cases}
$$

2. 次にscoreをsoftmax関数により**重み**$\ \boldsymbol{a}_t(s)$に変換します。
$$
    \boldsymbol{a}_t(s) = \frac{\exp\left[\mathrm{score}\left(\boldsymbol{\bar{h}}_s,\ \boldsymbol{h}_t\right)\right]}{\sum^S_{s'=1}\exp\left[\mathrm{score}\left(\boldsymbol{\bar{h}}_s,\ \boldsymbol{h}_t\right)\right]}
$$
3. 2.で計算した重みを元に、Encoderの出力の加重平均ベクトル (**文脈ベクトル**) $\boldsymbol{c}_t$ を計算します。
$$
    \boldsymbol{c}_t = \sum^S_{s=1} \boldsymbol{a}_t(s) \boldsymbol{\bar{h}}_s
$$
4. 3.で計算した文脈ベクトル$\boldsymbol{c}_t$と元の出力$\boldsymbol{h}_t$から、新しい出力ベクトル$\boldsymbol{\tilde{h}}_t$を計算します。
$$
    \boldsymbol{\tilde{h}}_t = \tanh\left(\boldsymbol{W}_c \left[\begin{array}{c} \boldsymbol{c}_t \\ \boldsymbol{h}_t \end{array}\right] + \boldsymbol{b}\right)
$$

論文では$\boldsymbol{\tilde{h}}_t$が次のタイムステップのLSTMにfeedされる方法も示されており、他の論文でもそのようにしているものも多いです。

その場合、Attention層とLSTM層は分けずに、同じクラスで書くことになります。

#### Attentionのマスクについて

先程と同様、ミニバッチ化の際には短い系列に対してpaddingを行いますが、Encoderのpadding部分にはAttentionの適用を回避したいわけです。

そこで、padding部分は$\exp\left[\mathrm{score}\left(\boldsymbol{\bar{h}}_s,\ \boldsymbol{h}_t\right)\right]$ が0になるように (score関数の値がとても小さな値になるように) マスクをかけます。

#### Attentionを用いたDecoderの定義

In [ ]:
class AttentionDecoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, batch_size, n_layers, drop_prob, padding_idx):
        super(AttentionDecoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.drop_prob = drop_prob
        self.padding_idx = padding_idx

        self.embed = nn.Embedding(output_size, embed_size, padding_idx=padding_idx)
        self.dropout = nn.Dropout(drop_prob)

        self.fc_hidden = nn.Linear(hidden_size, hidden_size, bias=False)
        self.fc_encoder = nn.Linear(hidden_size, hidden_size, bias=False)
        self.weight = nn.Parameter(torch.FloatTensor(batch_size, hidden_size))
        self.attn_combine = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(drop_prob)

        self.lstm = nn.LSTM(embed_size, hidden_size, n_layers)
        # LSTMの隠れ状態とAttention層で計算されるコンテキストベクトルを
        # 連結して入力するためhidden_size+hidden_sizeとしている。
        self.fc_out = nn.Linear(hidden_size + hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden, cell, encoder_output, mask):
        batch_size = input.shape[0]
        # Decoderへの入力をEmbedding
        embedded = # WRITE ME
        
        # dropoutを適用
        # embedded shape: [1, batch_size, encoder_embed_dim]
        embedded = # WRITE ME

        # Alignment Scoreを計算
        # scores shape: [batch_size, hidden_size, 1]
        scores = self.fc_hidden(hidden[0]).unsqueeze(2)
        # encoder_outputの順番を入れ替えてdot積を計算する: 
        # [sentence_length, batch_size, hidden_size] 
        # -> [batch_size, sentence_length, hidden_size]
        alignment_scores = torch.bmm(encoder_output.permute(1, 0, 2), scores)

        # Alignment ScoreにSoftmax関数を適用してAttentionの重みを取得する
        # masked_alignment_scores shape: [batch_size, sentence_length]
        masked_alignment_scores = alignment_scores.squeeze(2)*mask

        # attn_weights shape: [batch_size, sentence_length]
        attn_weights = F.softmax(masked_alignment_scores, dim=1)

        # Encoderの出力結果にAttentionの重みを適用しContext Vectorを取得
        # context_vector shape: [bathc_size, hidden_dim, 1]
        context_vector = torch.bmm(encoder_output.permute(1, 2, 0), attn_weights.unsqueeze(2))
        
        # LSTMにembeddedを入力し、隠れ状態を取得
        _, (hidden, cell) = # WRITE ME
        # LSTMの隠れ状態とContext Vectorを結合
        # hidden: [1, batch_size, hidden_size]
        # context_vector: [batch_size, hidden_size, 1]
        output = torch.cat((hidden.squeeze(0), context_vector.squeeze(2)), dim=1)
        
        # prediction shape: # [batch_size, output_size]
        prediction = self.fc_out(output.squeeze(0)) 

        return prediction, hidden, cell, attn_weights

In [ ]:
# 定義したEncoderとDecoderを用いてSeq2Seqモデルを定義する
class Seq2SeqAttention(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        src_text = src[0]  # Sourceのテキストを取得
        src_length = src[1]  # Sourceのテキスト長を取得
        trg_text = trg[0]  # Targetのテキストを取得
        trg_length = trg[1]  # Targetのテキスト長を取得
        batch_size = trg_text.shape[1]  # Batch Sizeを取得
        max_trg_len = trg_text.shape[0]  # Targetの最大文字列長を取得
        trg_vocab_size = self.decoder.output_size  # TargetのVocaburay数を取得

        # Encoderの最後の隠れ状態をDecoderの初期値とする
        encoder_output, encoder_hidden, encoder_cell, sent_len = self.encoder(src_text, src_length)
        
        # encoderのpadding文字列に対応するためのmask
        mask = torch.zeros(batch_size, max(sent_len)).to(DEVICE)
        for i in range(batch_size):
            pad_idx = sent_len[i]
            mask[i, :pad_idx] = 1

        # Decoderの出力結果を保持する配列
        outputs = torch.zeros(max_trg_len, batch_size, trg_vocab_size).to(DEVICE)
        max_src_len = encoder_output.shape[0]
        attentions = torch.zeros(max_trg_len, batch_size, max_src_len).to(DEVICE)

        # 文章の始まりを表す<sos>トークンをDecoderに入力する
        input = trg_text[0, :]

        # 各ステップを実行
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell
        for step in range(1, max_trg_len):
            # Attentionを実装したDecoderに入力する
            output, decoder_hidden, decoder_cell, decoder_attention = \
                self.decoder(input, decoder_hidden, decoder_cell, encoder_output, mask)
            outputs[step] = output
            attentions[step] = decoder_attention

            # 予測結果の配列から、もっとも確率が高い予測トークンを取得する
            top1 = # WRITE ME
            
            # 現在のステップでteacher forcingを用いるかどうか
            teacher_force = # WRITE ME

            # techer forceを用いる場合はtarget配列から該当する値を抽出し、
            # 用いない場合は予測結果を次のステップでの入力とする
            # WRITE ME

        return outputs, attentions

### 2. 学習

#### 学習用の関数

In [ ]:
def train_attention_model(model, loss_function, optimizer, num_epochs=5, clip=1.0):
    # 学習モードに設定
    model.train()

    for epoch in range(num_epochs):
        train_loss = 0.0
        for batch in tqdm(train_iter):
            optimizer.zero_grad()  # 勾配の初期化
            output, _ = model(batch.source, batch.target)  # 予測の計算（順伝播）
            
            # 予測結果の整形と正解データの抽出
            output_size = output.shape[-1]
            output = output[1:].view(-1, output_size)
            trg_text = batch.target[0]
            trg = trg_text[1:].view(-1)
            
            loss = loss_function(output, trg)  # 損失関数の計算
            train_loss += loss.item()  # 損失の加算
            loss.backward()  # 勾配の計算(逆伝播)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  # Gradient Clipping
            optimizer.step()  # パラメータの更新
            
        avg_train_loss = train_loss / len(train)  # 平均損失の計算
        print(('Epoch [{}/{}]: train_loss: {train_loss:.5f}')
          .format(epoch+1, num_epochs, train_loss=avg_train_loss))
        
    return model

#### 評価用の関数

In [ ]:
def evaluate_attention_model(model, loss_function):
    model.eval()  # 推論モードに切替
    with torch.no_grad():  # 計算グラフの構築をしないよう設定
        test_loss = 0
        for batch in tqdm(val_iter):
            output, _ = model(batch.source, batch.target, teacher_forcing_ratio=0.0)  # 予測の計算（順伝播）

            # 予測結果の整形と正解データの抽出
            output_size = output.shape[-1]
            output = output[1:].view(-1, output_size)
            trg_text = batch.target[0]
            trg = trg_text[1:].view(-1)

            loss = loss_function(output, trg)  # 損失関数の計算
            test_loss += loss.item()

        avg_test_loss = test_loss / len(val)
        print('Test Loss: {test_loss:.5f}'.format(test_loss=avg_test_loss))  

#### モデルの学習と評価

In [ ]:
input_size = len(SOURCE.vocab)
output_size = len(TARGET.vocab)
encoder_embed_size = 256
decoder_embed_size = 256
hidden_size = 512
n_layers = 1
encoder_dropout = 0.5
decoder_dropout = 0.5
src_padding_idx = SOURCE.vocab.stoi["<pad>"]
trg_padding_idx = TARGET.vocab.stoi["<pad>"]

# EncoderクラスとDecoderクラスを生成
encoder = Encoder(# WRITE ME)
attention_decoder = AttentionDecoder(# WRITE ME)

# デバイスを指定して翻訳モデルを生成
attention_model =# WRITE ME
optimizer =  # WRITE ME
loss_function =  # WRITE ME

# モデルのパラメータを初期化
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
attention_model.apply(init_weights)

# モデルの学習（train_attention_model関数を利用）
attention_model =  # WRITE ME

# モデルの評価（evaluate_attention_model関数を利用）
# WRITE ME

### 3. 翻訳文の生成とattention weightの可視化

### Attention weightの可視化

`matplotlib`で`Attention weight`を可視化してみましょう。

具体的には、訳文の生成を行う際に、原文のどの単語をどれだけ重視したかをヒートマップで可視化します。

アテンションで獲得した重み$a = [\boldsymbol{a}_1(s)\ \ \boldsymbol{a}_2(s)\ \ \cdots\ \ \boldsymbol{a}_T(s)]$を可視化してみます。

縦軸に訳文 (日本語)、横軸に原文 (英語)を表示し、重みをヒートマップで表示します。

In [ ]:
def plot_attention_weight(attention, source_text, predict_text):
    # Attentionのヒートマップを作成
    atten_matrix = attention.cpu().detach().numpy()  # [target_length, sentence_length]
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot()
    cax = ax.matshow(atten_matrix)
    fig.colorbar(cax)

    # 軸の設定
    ax.set_xticklabels([""]+source_text.split(" "), rotation=90)
    ax.set_yticklabels([""]+predict_text.split(" "))

    # ラベルを設定
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    plt.show()

In [ ]:
# ID変換された各単語のリストを文章に変換する
def token2text(tokens, VOCAB):
    texts = []
    for token in tokens[1:]:  # <sos>を除く
        text = VOCAB.vocab.itos[token]
        if text != "<eos>":
            texts.append(text)
        else:
            break       
    return " ".join(texts), len(texts)

# 出力例を確認
attention_model.eval()  # 推論モードに切替
batch = next(iter(val_iter))  
predicts, attentions = attention_model(batch.source, batch.target, teacher_forcing_ratio=0.0)  # 予測の計算
predicts = predicts.argmax(2)  # 予測値のみを取得
for i in range(5):
    source = batch.source[0][:, i]
    target = batch.target[0][:, i]
    source_text, source_len = token2text(source, SOURCE)
    target_text, target_len = token2text(target, TARGET)
    predict_text, predict_len = token2text(predicts[:, i], TARGET)
    print("---------------------------------------------------------------")
    print("English (input): {}".format(source_text))
    print("Japanese (answer): {}".format(target_text))
    print("predicted label: {}\n".format(predict_text))

    # plot_attention_weight関数を用いてAttentionの重みを確認
    # attentions: [max_predict_length, batch_size, max_source_length]
    # WRITE ME

---

### 【発展】 自然言語処理における最新の手法

最後に自然言語処理の分野で近年注目が集まってきている`BERT (Bidirectional Encoder Representations from Transformers)`と`GPT-3`について紹介します。

`BERT`と`GPT-3`ともに、`Transformer`と呼ばれるモデルがベースとなっています。

そのため、まずはTransformerについて紹介します。

## ・Transformerについて

`Transformer`は、先程紹介したSequence Modelの一つですが、RNNやCNNを利用せずAttentionのみを用いたモデルである点が特徴的です。そのため、RNNのように逐次的にモデルを学習する必要がなく並列化が容易なため計算時間を大きく削減できるといったメリットもあります。

Transformerは以下の図のような構造をしており、左側がEncoderで右側がDecoderとなっています。論文中では、EncoderとDecoderともにN=6層となっており、Decoder側にはEncoderの出力を受け取る`Multi-Head Attention`が中間に入っています。Decoder側の一つ目(一番下)のAttentionは、`Masked Multi-Head Attention`となっており、その他のAttentionと異なります。これは、Decoderに未来の情報が漏れないようにするためにMask処理を行ったもので、その他の点については`Multi-Head Attention`と同じ構造をしています。

<img src="./figures/transformer.png" width="1000mm">

Transformerの構造で特に重要な要素は、`Scaled Dot-Product Attention`、`Multi-Head Attention`と`Positional Encoding`の3つになります。

<img src="./figures/transformer_attention.png" width="550mm">


- **Scaled Dot-Product Attention**

  - 計算式は以下で与えられており、$ \sqrt{d_{k}} $で表されるスケール項が存在する以外は、課題3で紹介したものと大きく変わりません。式中の$QK^{T}$をスコア関数と捉えて解釈してください。
  - $ \sqrt{d_{k}} $がスケーリング項として作用します。$d_{k}$の値が大きい時は、Softmax関数の勾配がとても小さくなり、Dot-Product Attentionがうまく機能しないと考えられており、その調整のためにスケーリングを行っています。
  - 図中のMaskは、Decoderに未来の情報がリークしないように機能します。
  
$$ Attention(Q, K, V) = softmax(\frac{QK^{T}}{\sqrt{d_{k}}})V $$


- **Multi-Head Attention**

  - Multi-Head AttentionとはScaled Dot-Product Attentionを一つのHeadとして、複数のHeadを並列化したAttentionとなっています。
  - 複数のHeadからの出力を結合し、線形層に入力することで最終的なAttentionの結果を計算します。


- **Positional Encoding**

  - Transformerは、再帰 (recurrent)や畳み込み (convolution)を利用していないため、文章中の単語の位置 (順序)に関する情報を保持していません。そのため、`Positional Encoding`の行列 (PE)を入力に加算することで単語の位置情報を考慮できるようにしています。
  - PEは単語の位置によって一意な値を取るようになっています。論文中では、PEをsin波とcos波を用いて以下のように表現しています。`pos`は単語の位置を`i`は成分成分次元を表します。


$$ PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}}) $$
$$ PE_{(pos, 2i+1)} = cos(pos/10000^{2i/d_{model}}) $$


参考文献： Attention Is All You Need (https://papers.nips.cc/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html)

### モデルの定義

PyTorchでは、`torch.nn.Transformer`でTransformerを利用することができます。では、実際に先程の翻訳データセットに対して`Transformer`を活用してみます。

torch.nn.Transformerは、以下の9つの引数を取ります。
- **d_model**: encoder, decoderへ入力される特徴量の次元 (default=512)
- **nhead**: multi-head Attentionで利用するHead数 (default=8)
- **num_encoder_layers**: encoderの数 (default=6)
- **num_decoder_layers**: decoderの数 (default=6)
- **dim_feedforward**: feed foward networkの次元 (default=2048)
- **dropout**: ドロップアウト率 (default=0.1)
- **activation**: 活性化関数 (default=relu)
- **custom_encoder**: 自分で作成したencoder (defalut=None)
- **custom_decoder**: 自分で作成したdecoder (defalut=None)


公式ドキュメントには、サンプルコードなども記載されているため詳しく知りたい方は確認してみてください。

`torch.nn.Transfomer`のドキュメント: https://pytorch.org/docs/master/generated/torch.nn.Transformer.html

In [ ]:
# Positional Encodingの項で説明した数式を実装し、入力系列に足し合わせるクラス
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Positional Encodingの数式にしたがって実装
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe) # peは最適化しないパラメータ

    def forward(self, x):
        # Positional Encodingの情報を足し合わせる
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    

# 翻訳を行うTransformerを実装したクラス
class Transformer(nn.Module):
    def __init__(self, src_input_size, trg_input_size, 
                 embed_size, src_padding_idx, trg_padding_idx):
        super().__init__()
        self.embed_size = embed_size
        self.src_padding_idx = src_padding_idx
        self.trg_padding_idx = trg_padding_idx
        self.output_size = trg_input_size
        
        # embedding層を定義
        self.src_embed = nn.Embedding(src_input_size, embed_size, padding_idx=src_padding_idx)
        self.trg_embed = nn.Embedding(trg_input_size, embed_size, padding_idx=trg_padding_idx)
        
        # positional encoding層を定義
        self.pos_encoder = PositionalEncoding(embed_size, max_len=src_input_size)
        
        # 今回は、レイヤー数などのパラメータを論文と同じ値に設定します
        self.transformer = nn.Transformer(
                                d_model=embed_size,
                                nhead=8, 
                                num_encoder_layers=6,
                                num_decoder_layers=6)
        self.linear = nn.Linear(embed_size, self.output_size)
        
    def forward(self, src, trg):
        # sourceとtraget系列を取得
        # output shape: (sequence_length, batch_size)
        src_text = src[0] 
        trg_text = trg[0]
        
        # 各系列をEmbeddingし、positional encodingを行う
        # output shape: (sequence_length, batch_size, embed_size)
        src_embedded = self.src_embed(src_text)
        src_embedded = self.pos_encoder(src_embedded)
        trg_embedded = self.trg_embed(trg_text)
        trg_embedded = self.pos_encoder(trg_embedded)
        
        # 未来の系列がリークすることを防ぐマスク
        # マスクを生成するメソッドは、デフォルトでnn.Transformerに用意されています
        # output shape: (sequence_length, batch_size, embed_size)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_text.shape[0]).to(DEVICE)
        
        # Transformerに入力し、出力結果を線形層に入力する
        # output shape: (sequence_length, batch_size, output_size)
        output = self.transformer(src_embedded, trg_embedded, tgt_mask=trg_mask)
        output = self.linear(output)
        
        return output

### 学習

モデルの学習と評価には、Seq2Seqモデルで利用した`train_seq2seq_model`と`evaluate_seq2seq_model`を用います。

In [ ]:
# ハイパーパラメータの設定
src_input_size = len(SOURCE.vocab)
trg_input_size = len(TARGET.vocab)
embed_size = 512
src_padding_idx = SOURCE.vocab.stoi["<pad>"]
trg_padding_idx = TARGET.vocab.stoi["<pad>"]

# Transformerを定義
transformer_model = Transformer(
    src_input_size, 
    trg_input_size, 
    embed_size, 
    src_padding_idx, 
    trg_padding_idx).to(DEVICE)

optimizer = optim.Adam(transformer_model.parameters(), lr=0.0003)
loss_function = nn.CrossEntropyLoss(ignore_index=trg_padding_idx)

# モデルのパラメータを初期化   
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.01, 0.01)
transformer_model.apply(init_weights)

# モデルの学習（train_seq2seq_model関数を再利用）
# 1epochに、1分程度時間がかかります。
# num_epochs=20で比較的納得感のある翻訳モデルを作成することができますが、
# 動作確認で十分の場合は、epoch数を減らしてみてください。
transformer_model = train_seq2seq_model(
    transformer_model, loss_function, optimizer, num_epochs=20)

# モデルの評価（evaluate_seq2seq_model関数を再利用）
evaluate_seq2seq_model(transformer_model, loss_function)

### 予測結果の確認

トレーニングデータで学習した`Transformer`モデルを用いて、テストデータに対する出力結果を確認してみます。
出力結果の確認は、Seq2Seqモデルで利用した`print_seq2seq_result`を利用します。

In [ ]:
# 出力例を確認（print_seq2seq_result関数を再利用）
print_seq2seq_result(transformer_model)

---

## ・BERTについて

`BERT`とは`Transformer`を複数用いて文脈を双方向に学習した`Encoder`モデルです。

ベンチマークとして用いられている自然言語処理のタスク11個において、`SOTA (State of The Art)`を達成し、今までのスコアを大きく更新しました。加えて、事前学習済みの`BERT`を用いてファインチューニングすることで様々なタスクに対して、簡単に適用できるといった点や今までのモデルと比べて汎用性が高い点など実利用での利点もあります。

`BERT`の基本的な構造は図のようになっており、`Trm`と書かれた青色の丸が上記で紹介した`Transformer`を表しています。
具体的な内容までは今回の講義で扱いませんが、`BERT`は以下の2つのタスクで学習を行っています。

#### 1. Masked Language Model
`Masked Language Model`は穴埋め問題を解くようなタスクです。例えば、以下のような文章があったときにランダムに`Mask`をかけて前後の文脈から、その`Mask`された単語を予測します。

```
私は犬を連れて海辺を散歩しました。　→　私は犬を連れて [Mask] を散歩しました。
```


#### 2. Next Sentence Prediction
`Next Sentence Prediction`は2つの文章を入力として与え、その2つの文章に関係性があるかどうかを判断するタスクです。学習時には、入力される2つの文章において後半の文章がランダムに50%の確率で他の文章に置換されます。その文章ペアから前半の文章をもとに、後半の文章が関連性があるかどうかを判定していきます。


```
入力：　今日はとても天気がよかった。(前半)/ 溜まっていた洗濯物を干した。(後半)
出力：関係性あり

入力：今日はとても天気がよかった。(前半)/ 仕事が無事に終わった。(後半、置換)
出力：関係性なし
```

<img src="./figures/bert.png" width="300mm">

参考文献： BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (https://arxiv.org/abs/1810.04805)

`BERT`の実装については`Tensorflow`を用いたものが`Google`から公式に公開されています。しかし、2020年6月現在では、`PyTorch`版は公開されていません。

`PyTorch`を用いた`BERT`では、`HuggingFace`が公開している実装が一般的に用いられています。こちらでは様々な言語で事前学習したモデルが用意されています。日本語で事前学習したモデルとしては、東北大学の乾研究室で作成された以下の四つが用意されています。
1. bert-base-japanese
2. bert-base-japanese-whole-word-mask
3. bert-base-japanese-char
4. bert-base-japanese-char-whole-word-masking

今回は例としてChapter07で行った`Sentiment Analysis`を日本語で事前学習された`BERT`を用いて行ってみます。

まず、`pip`経由で必要なライブラリをインストールします。


PyTorch版BERT: https://github.com/huggingface/transformers  
各種事前学習モデルについて: https://huggingface.co/transformers/pretrained_models.html

In [ ]:
# 日本語で事前学習したBERTの動作確認
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
print("Tokenizerの単語数: {0}".format(len(tokenizer.vocab)))

# 文章をtokenに分割
text = "幅広い領域のデータが集まり、AIの活用範囲が広がった。"
tokens = tokenizer.tokenize(text)
print("例: {0}".format(tokens))

# IDに変換
indexs = tokenizer.convert_tokens_to_ids(tokens)
print("ID: {0}".format(indexs))

上記の`text`の値を変更してみて、正しく単語の分割が行われているか確認してみてください。

In [ ]:
# WRITE ME

### データセットの準備

まず、Chapter07で用いたSentiment Analysisのデータセットを準備します。

In [ ]:
def bert_tokenizer(sentence):
    tokens = tokenizer.tokenize(sentence)
    # 文章の開始を表す<sos>と終了を表す<eos>がsentenceに追加する必要があるので-2をしている
    tokens = tokens[:model_max_len-2]
    return tokens 


# 各種特殊文字のindexを取得
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

# モデルで扱える最大文字列長を取得
model_max_len = tokenizer.max_model_input_sizes["cl-tohoku/bert-base-japanese"]

# データセットを定義
TEXT = Field(batch_first=True, sequential=True, include_lengths=True,
              use_vocab=False, tokenize=bert_tokenizer, 
              preprocessing=tokenizer.convert_tokens_to_ids,
              init_token=init_token_idx,
              eos_token=eos_token_idx,
              pad_token=pad_token_idx,
              unk_token=unk_token_idx)
LABEL = LabelField(dtype = torch.long)


# 各種データの読み込み
train, val, test = TabularDataset.splits(
    path="/root/userspace/public/day4/chap09/data/",
    train="sentiment_train.csv", validation="sentiment_val.csv",
    test="sentiment_test.csv", format="csv",
    fields=[("text", TEXT), ("label", LABEL)]
)

# Labelを数字に変換
LABEL.build_vocab(train)

# イテレータの作成
batch_size = 16
train_iter, val_iter, test_iter = BucketIterator.splits(
    (train, val, test), batch_size=batch_size, device=DEVICE,
    sort=False
)

### モデルの定義

In [ ]:
class BERTSentiment(nn.Module):
    def __init__(self, bert, output_dim):
        super(BERTSentiment, self).__init__()
        self.bert = bert
        self.output_dim = output_dim

        embedding_dim = bert.config.to_dict()["hidden_size"]
        self.linear1 = nn.Linear(embedding_dim, 256)
        self.linear2 = nn.Linear(256, output_dim)
        self.act = nn.Sigmoid()


    def forward(self, inputs):
        # inputs = (text, text_length)
        inputs_text = inputs[0]  # 入力テキストを取得、 inputs_text = [batch_size, sentence lengths]

        # BERTをEncoderとして用いる
        # embedded shape: [batch_size, embedding_dim]
        _, embedded = self.bert(inputs_text)
        
        # embeddingの結果をlinear1に入力し、出力結果に活性化関数reluを適用する
        # output shape: [batch_size, 128]
        output = F.relu(self.linear1(embedded))
        
        # linear1の結果をlinear2に入力する
        # output shape: [batch_size, output_dim]
        output = self.linear2(output)
        
        # 出力をSigmoid関数で変換する
        # output shape: [batch_size]
        output = self.act(output.squeeze(1))
        
        return output

### 学習

モデルの学習と評価には、今までと同様に`train_model`と`evaluation`を用います。

In [ ]:
# パラメータの設定
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
output_dim = 1

# モデル定義
bert_model = BERTSentiment(bert, output_dim).to(DEVICE)
loss_function = nn.BCELoss().to(DEVICE)
optimizer = optim.Adam(bert_model.parameters(), lr=3e-5)

# モデルの学習
bert_model = train_model(bert_model, loss_function, optimizer, num_epochs=3)

# モデルの評価
evaluation(bert_model)

### 予測結果の確認

トレーニングデータで学習した`BERT`モデルを用いて、テストデータに対する出力結果を確認してみます。

In [ ]:
def token2text(tokens):
    texts = ""
    for token in tokens:
        if token != tokenizer.sep_token_id:
            texts += " {}".format(tokenizer.convert_ids_to_tokens(token.item()))
        else:
            break
    return texts

# 出力例を確認
bert_model.eval()  # 推論モードに切替
batch = next(iter(val_iter))  
predicts = bert_model(batch.text)  # 予測の計算
predicts = torch.round(predicts)  # 予測値をラベルに変換
for i in range(5):
    tokens = batch.text[0][i, :]
    tokens = tokens[1:]  # 開始を表す記号を除く
    print("input text: {}".format(token2text(tokens)))
    print("answer label: {}".format(LABEL.vocab.itos[batch.label[i]]))
    print("predicted label: {}\n".format(LABEL.vocab.itos[int(predicts[i])]))

---

## GPT-3について

`GPT-3`は、人工知能に関する研究を行う非営利団体である[OpenAI](https://openai.com/)から2020年に発表された言語生成モデルです。

`GPT-3`の内部構造は、先ほど紹介した`BERT`と同様に`Transfomer`をベースに構築されています。しかし、ニューラルネットワークのパラメータ数と学習データの量が大きく増加しています。

`GPT-3`のパラメータ数は1750億個となっており、`BERT`の3.4億と比べかなり大きなモデルになっていることがわかります (`GPT-3`の前バージョンである`GPT-2`では、パラメータ数が15億個でした)。また、事前学習に利用したデータ量に関しても`GPT-3`が45TBで、`BERT`が16GBと約2800倍となっています。パラメータ数とモデルの精度の関係について、理論的な裏付けがあるわけではありませんが、論文中ではパラメータ数を変更しモデルの規模を調整した場合の比較も行われています。結果としては、1750億個のパラメータを使用した、もっとも大きなモデルが高い精度を記録しました。

`GPT-3`は、論文のタイトルにあるように`Few-Shot Learning`でモデルの学習を行っています。`Few-Shot Learning`の説明を行うために、`Zero-Shot Learning`と`One-Shot Learning`の説明を行います。詳しい説明については、論文中にも記載があるので興味がある方は確認してみてください。

- **Zero-Shot Learning**

  - どのようなことを行うのかを表すタスクの説明とクエリのみを入力します。
  - 図の場合だと、`"Translate English to French:\n cheese => "`が入力となります。
 
<img src="./figures/zero_shot.png" width="550mm">


- **One-Shot Learning**

  - どのようなことを行うのかを表すタスクの説明と一つの例、そしてクエリを入力します。
  - 図の場合だと、`"Translate English to French:\n sea otter => loutre de mer \n cheese =>"`が入力となります。
  
<img src="./figures/one_shot.png" width="550mm">


- **Few-Shot Learning**

  - どのようなことを行うのかを表すタスクの説明といくつかの例、そしてクエリを入力します。
  - 図の場合だと、`"Translate English to French:\n sea otter => loutre de mer \n peppermint => menthe poivree \n plush girafe => girafe peluche \n cheese =>"`が入力となります。
  
<img src="./figures/few_shot.png" width="550mm">



#### 活用例

`GPT-3`はプライベートなベータ版のAPIが[公式サイト](https://beta.openai.com/?app=productivity)から公開されていますが、2020年12月現在では順番待ちのリストに申請する必要があります。実際に申請が通過した研究者などが、自然言語を入力としWebサイトを構築するプログラムを生成した場合の利用結果などを公開しているので確認してみると面白いと思います。また、[公式サイト](https://beta.openai.com/?app=productivity)でもいくつかサンプルが公開されています。


参考文献： Language Models are Few-Shot Learners (https://arxiv.org/abs/2005.14165)